In [2]:
import pandas as pd
import os
from pathlib import Path
from hackutilitiy import *
import matplotlib.pyplot as plt

## Report on text data

In [ ]:
# Read all .csv files to dataframes. For larger files read a chunk for a sneak peek.
data_dct = csv_to_dataframe(nrows=100_000) 

# Read the factor descriptions provided in the instructions manual to dataframe
factor_glossary = pd.read_csv('./table1/table1.csv')

factor_features = data_dct['factor_char_list']['variable'].tolist()
non_factor_features = [c for c in (set(data_dct['ret_sample'].columns) - set(factor_features))]

# Distinguish referenced and non referenced factors
factors_no_ref = list(set(factor_features)-set(factor_glossary["Acronym"].values))
factors_with_ref = list(set(factor_features).intersection(set(factor_glossary["Acronym"].values)))

# Accounting ratios in acc_ratio.csv that can serve as targets.
acc_ratios = data_dct['acc_ratios']['Variable'].tolist()
# Unreferenced accounting factors
unreferenced_accounting_factors = list(set(acc_ratios).intersection(set(factors_no_ref)))

# List refrenced factors
idx_factor_features = factor_glossary.index[factor_glossary["Acronym"].isin(factors_with_ref)].tolist()
factor_glossary['Feature'].iloc[idx_factor_features].reset_index().drop('index', axis=1).rename(columns={'Feature':'Referenced Factors'})

## Report on text data

In [ ]:
# Read the .pkl file
txt_sample = pd.read_pickle("TEXT DATA US by YEAR/raw_text_hack_2025/text_us_2025.pkl")
txt_sample['date'] = pd.to_datetime(txt_sample.date)


In [53]:
# Display basic information about the DataFrame
txt_sample.info()  # Shows column names, non-null counts, and memory usage

<class 'pandas.core.frame.DataFrame'>
Index: 20352 entries, 531900 to 555999
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       20352 non-null  datetime64[ns]
 1   cik        20352 non-null  int64         
 2   file_type  20352 non-null  object        
 3   rf         20352 non-null  object        
 4   mgmt       20352 non-null  object        
 5   gvkey      17989 non-null  float64       
 6   cusip      17989 non-null  object        
 7   year       20352 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 1.4+ MB


In [55]:
txt_sample = txt_sample.dropna().astype({"gvkey": int})
txt_sample.tail()


,date,cik,file_type,rf,mgmt,gvkey,cusip,year
555970,2024-12-27,1788230,10Q,Item\n 1A.\nRisk\n Factors\n \n70 Item 1...,Item\n 2.\nManagement’s\n Discussion and...,45332,42727R104,2024
555978,2024-12-27,1851484,10K,Item 1A.\nRisk Factors\n17 Item 1A. Risk Facto...,Item 7.\nManagement’s Discussion\n and Anal...,38840,17331Y109,2024
556002,2024-12-31,72633,10K,Item 1A. \nRisk Factors.\nNot applicable.,Item 7.\nManagement's Discussion and Analysis ...,7959,659310106,2024
555997,2024-12-31,783412,10K,Item 1A. Risk Factors\n \nThe foregoing busine...,Item 7. Management's Discussion and Analysis o...,12568,233912104,2024
556000,2024-12-31,1360442,10Q,Item 1A. Risk Factors\n \nNot required.,Item 2. Management’s Discussion and Analysis o...,18387,13764T105,2024


In [ ]:
txt_sample.isna().sum()  # Shows the count of missing values in each column 

date         0
cik          0
file_type    0
rf           0
mgmt         0
gvkey        0
cusip        0
year         0
dtype: int64

In [104]:
#Standard identifiers for alignment: gvkey or cusip (with year-month for time control); 
for k in data_dct.keys():
    if k !='ret_sample':
        print("###", k,"columns", list(data_dct[k].columns),"\n", "dtypes:\n", data_dct[k].dtypes)

### factor_char_list columns ['variable'] 
 dtypes:
 variable    object
dtype: object
### mkt_ind columns ['rf', 'year', 'month', 'ret'] 
 dtypes:
 rf       float64
year       int64
month      int64
ret      float64
dtype: object
### acc_ratios columns ['Variable', 'Description'] 
 dtypes:
 Variable       object
Description    object
dtype: object
### na_name_merge columns ['gvkey', 'date', 'iid', 'tic', 'cusip', 'conm', 'cik'] 
 dtypes:
 gvkey             int64
date     datetime64[ns]
iid              object
tic              object
cusip            object
conm             object
cik             float64
dtype: object
### global_name_merge columns ['fic', 'gvkey', 'date', 'iid', 'conm'] 
 dtypes:
 fic              object
gvkey             int64
date     datetime64[ns]
iid              object
conm             object
dtype: object
### us_cik_gvkey_linktable columns ['gvkey', 'date', 'iid', 'tic', 'cusip', 'conm', 'tpci', 'cik'] 
 dtypes:
 gvkey             int64
date     datetime64[ns]
ii

In [115]:
# Consolidate the meta data by collecting tickers, companies, geography, etc. from the link tables:
txt_db = txt_sample.loc[:,["year", "date", "file_type", "rf", "mgmt", "gvkey", "cusip", "cik" ]].copy()
txt_db.tail()

,year,date,file_type,rf,mgmt,gvkey,cusip,cik
213523,2006,2006-12-29,10K,ITEM 1A. \n\nRISK\n FACTORS \n\n13 Item 1A. \n...,ITEM 7. \n\nMANAGEMENT S\n DISCUSSION AND ANAL...,113361,149016107,1063085
213506,2006,2006-12-29,10K,Item\n 1A. \n\nRisk\n Factors \n\n1.\n We May ...,Item\n 7. \n\nManagement s\n Discussion and An...,147249,47077R109,1133062
213509,2006,2006-12-29,10K,ITEM 1A \n\nRISK FACTORS \n \n 11 ITEM 1A. Ris...,ITEM 7. \n\nMANAGEMENT S DISCUSSION AND ANALYS...,230796,00506P103,1183941
213524,2006,2006-12-29,10K,ITEM 1A. RISK FACTORS Risk Factors This report...,ITEM 7. MANAGEMENT S DISCUSSION AND ANALYSIS O...,162956,00430L103,1310094
213529,2006,2006-12-29,10K,Item 1A. Risk Factors 33 ITEM 1A. RISK FACTORS...,Item 7. Management's Discussion and Analysis o...,162576,114039100,1310313


In [93]:
data_dct['global_name_merge'].isna().sum()

fic      9482
gvkey       0
date        0
iid         0
conm     9482
dtype: int64

In [109]:
data_dct['global_name_merge'].isna().sum()

fic      9482
gvkey       0
date        0
iid         0
conm     9482
dtype: int64

In [114]:
txt_db.columns

Index(['year', 'date', 'file_type', 'rf', 'mgmt', 'gvkey', 'cusip', 'cik',
       'iid', 'tic', 'conm', 'tpci', 'fic'],
      dtype='object')

In [ ]:
# 1st thought: the iid may need to be found for the gvkey-cik-cusip-date combination in txt_db 
# 2nd thought: drop duplicates in the merge key to avoid many-to-one merge. all  duplicates have the same conm
txt_db.merge(data_dct['global_name_merge'].drop_duplicates(subset=["gvkey", "date"]),
              how='left', on =["gvkey",  "date"], validate="one_to_one").fic.dropna()

8498    PAN
Name: fic, dtype: object

In [69]:
# 1st thought: the iid may need to be found for the gvkey-cik-cusip-date combination in txt_db 
# 2nd thought: drop duplicates in the merge key to avoid many-to-one merge. all  duplicates have the same conm
txt_db.merge(data_dct['na_name_merge'].drop_duplicates(subset=["gvkey", "cik", "cusip", "date"]),
              how='left', on =["gvkey", "cik", "cusip", "date"]).dropna().drop(['cik', 'cusip', 'gvkey', 'iid'], axis=1).reset_index(drop=True)   

,year,date,file_type,rf,mgmt,tic,conm
0,2005,2005-01-31,10K,,ITEM\n 7 MANAGEMENT'S DISCUSSION AND ANALYSIS ...,CHMP,CHAMPION INDUSTRIES INC
1,2005,2005-01-31,10Q,,ITEM 2 \n\nMANAGEMENT S DISCUSSION AND ANALYSI...,HSH,HILLSHIRE BRANDS CO
2,2005,2005-01-31,10Q,,Item 2. Management's Discussion and Analysis o...,KNAP,KNAPE & VOGT MFG CO
3,2005,2005-01-31,10K,,Item 7 \n Management's Discussion and Analysis...,POWL,POWELL INDUSTRIES INC
4,2005,2005-01-31,10Q,,Item 2. Management's Discussion and Analysis o...,PG,PROCTER & GAMBLE CO
...,...,...,...,...,...,...,...
493,2005,2005-06-30,10K,,Item 7. \n \n MANAGEMENT S DISCUSSION AND \n\n...,HDLM,HANDLEMAN CO
494,2005,2005-06-30,10Q,,Item 2. Management's Discussion and Analysis o...,GTK,GTECH HOLDINGS CORP
495,2005,2005-06-30,10K,,Item 7 \n\nManagement s Discussion and Analysi...,ID.2,L-1 IDENTITY SOLUTIONS INC
496,2005,2005-06-30,10K,,Item 7. Management s Discussion and Analysis o...,HOPE,HOPE BANCORP INC


In [72]:
# 1st thought: the iid may need to be found for the gvkey-cik-cusip-date combination in txt_db 
# 2nd thought: drop duplicates in the merge key to avoid many-to-one merge. all  duplicates have the same conm
txt_db.merge(data_dct['us_cik_gvkey_linktable'].drop_duplicates(subset=["gvkey", "cik", "cusip", "date"]),
              how='left', 
              on =["gvkey", "cik", "cusip", "date"]).dropna().drop(['cik', 'cusip', 'gvkey', 'iid'], axis=1).reset_index(drop=True).iloc[300:400]  

,year,date,file_type,rf,mgmt,tic,conm,tpci
300,2005,2005-03-31,10K,,ITEM 7. Management's Discussion and Analysis o...,CMOH,COMMERCIAL BANCSHARES INC/OH,0
301,2005,2005-03-31,10K,,ITEM\n7. MANAGEMENT S DISCUSSION AND ANALYSIS\...,ZOMX,ZOMAX INC/MN,0
302,2005,2005-03-31,10K,,ITEM 7 -\nMANAGEMENT S DISCUSSION AND ANALYSIS...,ANS,AIRNET SYSTEMS INC,0
303,2005,2005-03-31,10KSB,,ITEM 6. MANAGEMENT'S DISCUSSION AND ANALYSIS O...,EPMD,EP MEDSYSTEMS INC,0
304,2005,2005-03-31,10K,Item 7. \n\nRisk Factors \n\n29,Item 7. \n\nManagement s Discussion and Analys...,ARDMQ,ARADIGM CORP,0
...,...,...,...,...,...,...,...,...
395,2005,2005-03-31,10K,,ITEM 7. MANAGEMENT'S DISCUSSION AND ANALYSIS O...,PRPL.1,PURPLE COMMUNICATIONS INC,0
396,2005,2005-03-31,10K,,Item 7 - Management's Discussion and Analysis ...,NBAN,NORTH BAY BANCORP/CA,0
397,2005,2005-03-31,10K,,Item 7. Management's Discussion and Analysis o...,BCONQ,BEACON POWER CORP,0
398,2005,2005-03-31,10K,,ITEM\n 7 \n\nMANAGEMENT S DISCUSSION AND ANALY...,OYSM,OASYS MOBILE INC,0


In [119]:
reports.keys()

dict_keys(['factor_char_list.csv', 'mkt_ind.csv', 'ret_sample.csv', 'acc_ratios.csv', 'North America Company Name Merge by DataDate-GVKEY-IID.csv', 'Global (ex Canada and US) Company Name Merge by DataDate-GVKEY-IID.csv', 'cik_gvkey_linktable_USA_only.csv'])

In [116]:
# first attempt at merge
for t in ['us_cik_gvkey_linktable', 'na_name_merge', 'global_name_merge']:
    # Decide the keys to merge on
    keys = list(set(txt_db.columns) & set(data_dct[t].columns)) # not correct! introduces new columns to base df thereafter used in subsequent merge!
    print(keys) 
    txt_db = txt_db.merge(data_dct[t].drop_duplicates(subset=keys).drop('iid', axis=1), on=keys, how="left", validate="one_to_one")
    txt_db.describe()

['gvkey', 'cusip', 'cik', 'date']
['conm', 'tic', 'cusip', 'gvkey', 'date', 'cik']
['conm', 'gvkey', 'date']


In [120]:
# Prepare the text data for feature engineering. Decide what to extract from the 10K and 10Q files for each company.
# Store the results in a vector data base
idtfiers = ['gvkey', 'cusip', 'cik']
txt_db.drop(idtfiers, axis=1).set_index(txt_db.date).drop("date", axis=1).loc[:, ["year", "file_type", "conm", "tic", "rf", "mgmt", "tpci"]]

,year,file_type,conm,tic,rf,mgmt,tpci
date,,,,,,,
2006-01-03,2006,10Q,NaN,NaN,,Item 2. \n Management's Discussion and Analysi...,NaN
2006-01-03,2006,10K,NaN,NaN,Item 1A. Risk Factors\n- -------- ------------...,Item 7. Management's Discussion and Analysis o...,NaN
2006-01-03,2006,10Q,NaN,NaN,,ITEM 2. \n\nMANAGEMENT S DISCUSSION AND ANALYS...,NaN
2006-01-04,2006,10Q,NaN,NaN,,Item 2. \n Management s Discussion and Analysi...,NaN
2006-01-04,2006,10Q,NaN,NaN,,Item 2. \n \n Management s Discussion and Anal...,NaN
...,...,...,...,...,...,...,...
2006-12-29,2006,10K,NaN,NaN,ITEM 1A. \n\nRISK\n FACTORS \n\n13 Item 1A. \n...,ITEM 7. \n\nMANAGEMENT S\n DISCUSSION AND ANAL...,NaN
2006-12-29,2006,10K,NaN,NaN,Item\n 1A. \n\nRisk\n Factors \n\n1.\n We May ...,Item\n 7. \n\nManagement s\n Discussion and An...,NaN
2006-12-29,2006,10K,NaN,NaN,ITEM 1A \n\nRISK FACTORS \n \n 11 ITEM 1A. Ris...,ITEM 7. \n\nMANAGEMENT S DISCUSSION AND ANALYS...,NaN


🔹 cik

Meaning: Central Index Key.

Issued by the SEC’s EDGAR system to uniquely identify companies and individuals who file disclosure documents with the SEC.

Example: Apple’s CIK = 0000320193.

Used to link Compustat firms (gvkey) with SEC filings (10-K, 10-Q).

🔹 tpci

Meaning: Trading Public Company Indicator (Compustat variable).

A flag indicating the trading status of the security or company.

Values (from WRDS / Compustat docs):

"0" = Not a public company / not trading

"1" = Publicly traded company

Sometimes extended values exist depending on dataset vintage.

🔹 In your us_cik_gvkey_linktable context

gvkey → Compustat’s unique firm identifier.

cik → SEC’s identifier (so you can join to EDGAR filings).

tpci → Indicator whether the gvkey is linked to a publicly traded company (helps filter out private or non-trading entities when consolidating).

✅ So practically:

Use cik when you want to pull SEC filings for the firm.

Use tpci to filter down to actively traded companies (exclude stale or private).




🔹 cusip

Meaning: Committee on Uniform Securities Identification Procedures number.

It’s a 9-character alphanumeric identifier assigned to U.S. and Canadian securities (stocks and bonds).

Structure:

First 6 = issuer code.

Next 2 = issue identifier.

Last 1 = check digit.

In Compustat: lets you link to market data (CRSP, Bloomberg, etc.) at the security level.

Example: Apple Inc. common stock = 037833100.

🔹 iid

Meaning: Issue Identifier (Compustat).

Distinguishes different securities (issues) for the same firm (gvkey).

Example: A company might have common stock, preferred stock, bonds — all share the same gvkey but different iid.

Together, (gvkey, iid) pinpoints a unique security within a firm.

🔹 fic

Meaning: Foreign Incorporation Code.

Indicates the country of incorporation of the firm.

It’s a 3-character ISO country code (e.g., USA, CAN, GBR).

Useful when consolidating across global datasets or filtering U.S. vs foreign companies.

✅ In summary, in this context:

cusip → security identifier, links to markets.

iid → security issue within the firm.

fic → firm’s country of incorporation.